In [9]:
!pip install wordcloud

In [ ]:
!pip uninstall absl-py

In [ ]:
!pip install absl-py

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io 
import re                                  # library for regular expression operations
import string                              # for string operations
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import regexp_tokenize   # module for tokenizing strings
from nltk.tokenize import TreebankWordTokenizer
import nltk 
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [65]:
train_df = pd.read_csv(r'C:\Users\Lenovo\Desktop\GP Code\returns.csv')# change path to csv file
train_df.head()

,1
0,0
1,0
2,0
3,0
4,0


# Clean Data


### Remove any empty row after cleaning

In [12]:
train_df['tweet'].replace('', np.nan, inplace=True)
train_df.dropna(subset=['tweet'], inplace=True)

### Remove @ username in Twitter

In [13]:
def remove_at_user(text):

    """
      Remove @username from tweets
    """
#     print(text)
    return re.sub("@[A-Za-z0-9]+","",str(text))

In [14]:
train_df["text_clean"] = train_df["tweet"].apply(lambda x: remove_at_user(x))

###  Contractions 

We use the contractions package to expand the contraction in English such as we'll -> we will or we shouldn't've -> we should not have.

In [15]:
!pip install contractions


  Using cached textsearch-0.0.21-py2.py3-none-any.whl (7.5 kB)
  Using cached anyascii-0.2.0-py3-none-any.whl (283 kB)
  Using cached pyahocorasick-1.4.2-cp38-cp38-win_amd64.whl


In [16]:
import contractions
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: contractions.fix(x))

In [17]:
print(contractions.fix("you don't deserve to exist "))

you do not deserve to exist 


### Repeat the captial words 
Here we repeat the Captial words to confirm the meaning of this word as the user try to focus on it


In [18]:
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
train_df['tokenized'] = train_df["text_clean"].apply(word_tokenize)
train_df.head()


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


,tweet,label,text_clean,tokenized
0,i don't like chatting with you.,0,i do not like chatting with you.,"[i, do, not, like, chatting, with, you, .]"
1,"you do not have friends, cause no body loves you.",1,"you do not have friends, because no body loves...","[you, do, not, have, friends, ,, because, no, ..."
2,you don't deserve to exist.,1,you do not deserve to exist.,"[you, do, not, deserve, to, exist, .]"
3,no one cares about your opinion.,1,no one cares about your opinion.,"[no, one, cares, about, your, opinion, .]"
4,no one respect you.,1,no one respect you.,"[no, one, respect, you, .]"


In [19]:
train_df['text_clean']=train_df['tokenized'].apply(lambda x:  [word  if word.isupper() == False else word+" "+word  for word in x]) 
train_df['text_clean'] = [' '.join(map(str, l)) for l in train_df['text_clean']] # join back to text

In [20]:
train_df.head()

,tweet,label,text_clean,tokenized
0,i don't like chatting with you.,0,i do not like chatting with you .,"[i, do, not, like, chatting, with, you, .]"
1,"you do not have friends, cause no body loves you.",1,"you do not have friends , because no body love...","[you, do, not, have, friends, ,, because, no, ..."
2,you don't deserve to exist.,1,you do not deserve to exist .,"[you, do, not, deserve, to, exist, .]"
3,no one cares about your opinion.,1,no one cares about your opinion .,"[no, one, cares, about, your, opinion, .]"
4,no one respect you.,1,no one respect you .,"[no, one, respect, you, .]"


###  Covert to lower case

In [21]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: x.lower())
display(train_df.head())

,tweet,label,text_clean,tokenized
0,i don't like chatting with you.,0,i do not like chatting with you .,"[i, do, not, like, chatting, with, you, .]"
1,"you do not have friends, cause no body loves you.",1,"you do not have friends , because no body love...","[you, do, not, have, friends, ,, because, no, ..."
2,you don't deserve to exist.,1,you do not deserve to exist .,"[you, do, not, deserve, to, exist, .]"
3,no one cares about your opinion.,1,no one cares about your opinion .,"[no, one, cares, about, your, opinion, .]"
4,no one respect you.,1,no one respect you .,"[no, one, respect, you, .]"


### Remove any URLS

In [22]:
def remove_URL(text):
    """
        Remove URLs from a sample string
    """

    return re.sub(r"https?://\S+|www\.\S+", "", text)

In [23]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_URL(x))

### Remove HTML Tages

In [24]:
def remove_html(text):
    """
        Remove the html in sample text
    """
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

In [25]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_html(x))

### Remove Numbers

In [26]:
def Remove_numbers(text):

    """
      Remove the numbers in text 
    """
    result = ''.join(i for i in text if not i.isdigit())
    return result

In [27]:
s =  "i love you 10000000000 times"
print(Remove_numbers(s))

i love you  times


In [28]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: Remove_numbers(x))

### Replace the Typos, slang, acronyms or informal abbreviations

In [29]:
def informalAbbreviations_clean(text):
        """
            Other manual text cleaning techniques
        """
        # Typos, slang and other
        sample_typos_slang = {
                                "w/e": "whatever",
                                "usagov": "usa government",
                                "recentlu": "recently",
                                "ph0tos": "photos",
                                "amirite": "am i right",
                                "exp0sed": "exposed",
                                "<3": "love",
                                "luv": "love",
                                "amageddon": "armageddon",
                                "trfc": "traffic",
                                "16yr": "16 year"
                                }

        # Acronyms
        sample_acronyms =  { 
                            "mh370": "malaysia airlines flight 370",
                            "okwx": "oklahoma city weather",
                            "arwx": "arkansas weather",    
                            "gawx": "georgia weather",  
                            "scwx": "south carolina weather",  
                            "cawx": "california weather",
                            "tnwx": "tennessee weather",
                            "azwx": "arizona weather",  
                            "alwx": "alabama weather",
                            "usnwsgov": "united states national weather service",
                            "2mw": "tomorrow"
                            }

        
        # Some common abbreviations 
        sample_abbr = {
                        "$" : " dollar ",
                        "€" : " euro ",
                        "4ao" : "for adults only",
                        "a.m" : "before midday",
                        "a3" : "anytime anywhere anyplace",
                        "aamof" : "as a matter of fact",
                        "acct" : "account",
                        "adih" : "another day in hell",
                        "afaic" : "as far as i am concerned",
                        "afaict" : "as far as i can tell",
                        "afaik" : "as far as i know",
                        "afair" : "as far as i remember",
                        "afk" : "away from keyboard",
                        "app" : "application",
                        "approx" : "approximately",
                        "apps" : "applications",
                        "asap" : "as soon as possible",
                        "asl" : "age, sex, location",
                        "atk" : "at the keyboard",
                        "ave." : "avenue",
                        "aymm" : "are you my mother",
                        "ayor" : "at your own risk", 
                        "b&b" : "bed and breakfast",
                        "b+b" : "bed and breakfast",
                        "b.c" : "before christ",
                        "b2b" : "business to business",
                        "b2c" : "business to customer",
                        "b4" : "before",
                        "b4n" : "bye for now",
                        "b@u" : "back at you",
                        "bae" : "before anyone else",
                        "bak" : "back at keyboard",
                        "bbbg" : "bye bye be good",
                        "bbc" : "british broadcasting corporation",
                        "bbias" : "be back in a second",
                        "bbl" : "be back later",
                        "bbs" : "be back soon",
                        "be4" : "before",
                        "bfn" : "bye for now",
                        "blvd" : "boulevard",
                        "bout" : "about",
                        "brb" : "be right back",
                        "bros" : "brothers",
                        "brt" : "be right there",
                        "bsaaw" : "big smile and a wink",
                        "btw" : "by the way",
                        "bwl" : "bursting with laughter",
                        "c/o" : "care of",
                        "cet" : "central european time",
                        "cf" : "compare",
                        "cia" : "central intelligence agency",
                        "csl" : "can not stop laughing",
                        "cu" : "see you",
                        "cul8r" : "see you later",
                        "cv" : "curriculum vitae",
                        "cwot" : "complete waste of time",
                        "cya" : "see you",
                        "cyt" : "see you tomorrow",
                        "dae" : "does anyone else",
                        "dbmib" : "do not bother me i am busy",
                        "diy" : "do it yourself",
                        "dm" : "direct message",
                        "dwh" : "during work hours",
                        "e123" : "easy as one two three",
                        "eet" : "eastern european time",
                        "eg" : "example",
                        "embm" : "early morning business meeting",
                        "encl" : "enclosed",
                        "encl." : "enclosed",
                        "etc" : "and so on",
                        "faq" : "frequently asked questions",
                        "fawc" : "for anyone who cares",
                        "fb" : "facebook",
                        "fc" : "fingers crossed",
                        "fig" : "figure",
                        "fimh" : "forever in my heart", 
                        "ft." : "feet",
                        "ft" : "featuring",
                        "ftl" : "for the loss",
                        "ftw" : "for the win",
                        "fwiw" : "for what it is worth",
                        "fyi" : "for your information",
                        "g9" : "genius",
                        "gahoy" : "get a hold of yourself",
                        "gal" : "get a life",
                        "gcse" : "general certificate of secondary education",
                        "gfn" : "gone for now",
                        "gg" : "good game",
                        "gl" : "good luck",
                        "glhf" : "good luck have fun",
                        "gmt" : "greenwich mean time",
                        "gmta" : "great minds think alike",
                        "gn" : "good night",
                        "g.o.a.t" : "greatest of all time",
                        "goat" : "greatest of all time",
                        "goi" : "get over it",
                        "gps" : "global positioning system",
                        "gr8" : "great",
                        "gratz" : "congratulations",
                        "gyal" : "girl",
                        "h&c" : "hot and cold",
                        "hp" : "horsepower",
                        "hr" : "hour",
                        "hrh" : "his royal highness",
                        "ht" : "height",
                        "ibrb" : "i will be right back",
                        "ic" : "i see",
                        "icq" : "i seek you",
                        "icymi" : "in case you missed it",
                        "idc" : "i do not care",
                        "idgadf" : "i do not give a damn fuck",
                        "idgaf" : "i do not give a fuck",
                        "idk" : "i do not know",
                        "ie" : "that is",
                        "i.e" : "that is",
                        "ifyp" : "i feel your pain",
                        "IG" : "instagram",
                        "iirc" : "if i remember correctly",
                        "ilu" : "i love you",
                        "ily" : "i love you",
                        "imho" : "in my humble opinion",
                        "imo" : "in my opinion",
                        "imu" : "i miss you",
                        "iow" : "in other words",
                        "irl" : "in real life",
                        "j4f" : "just for fun",
                        "jic" : "just in case",
                        "jk" : "just kidding",
                        "jsyk" : "just so you know",
                        "l8r" : "later",
                        "lb" : "pound",
                        "lbs" : "pounds",
                        "ldr" : "long distance relationship",
                        "lmao" : "laugh my ass off",
                        "lmfao" : "laugh my fucking ass off",
                        "lol" : "laughing out loud",
                        "ltd" : "limited",
                        "ltns" : "long time no see",
                        "m8" : "mate",
                        "mf" : "motherfucker",
                        "mfs" : "motherfuckers",
                        "mfw" : "my face when",
                        "mofo" : "motherfucker",
                        "mph" : "miles per hour",
                        "mr" : "mister",
                        "mrw" : "my reaction when",
                        "ms" : "miss",
                        "mte" : "my thoughts exactly",
                        "nagi" : "not a good idea",
                        "nbc" : "national broadcasting company",
                        "nbd" : "not big deal",
                        "nfs" : "not for sale",
                        "ngl" : "not going to lie",
                        "nhs" : "national health service",
                        "nrn" : "no reply necessary",
                        "nsfl" : "not safe for life",
                        "nsfw" : "not safe for work",
                        "nth" : "nice to have",
                        "nvr" : "never",
                        "nyc" : "new york city",
                        "oc" : "original content",
                        "og" : "original",
                        "ohp" : "overhead projector",
                        "oic" : "oh i see",
                        "omdb" : "over my dead body",
                        "omg" : "oh my god",
                        "omw" : "on my way",
                        "p.a" : "per annum",
                        "p.m" : "after midday",
                        "pm" : "prime minister",
                        "poc" : "people of color",
                        "pov" : "point of view",
                        "pp" : "pages",
                        "ppl" : "people",
                        "prw" : "parents are watching",
                        "ps" : "postscript",
                        "pt" : "point",
                        "ptb" : "please text back",
                        "pto" : "please turn over",
                        "qpsa" : "what happens", #"que pasa",
                        "ratchet" : "rude",
                        "rbtl" : "read between the lines",
                        "rlrt" : "real life retweet", 
                        "rofl" : "rolling on the floor laughing",
                        "roflol" : "rolling on the floor laughing out loud",
                        "rotflmao" : "rolling on the floor laughing my ass off",
                        "rt" : "retweet",
                        "ruok" : "are you ok",
                        "sfw" : "safe for work",
                        "sk8" : "skate",
                        "smh" : "shake my head",
                        "sq" : "square",
                        "srsly" : "seriously", 
                        "ssdd" : "same stuff different day",
                        "tbh" : "to be honest",
                        "tbs" : "tablespooful",
                        "tbsp" : "tablespooful",
                        "tfw" : "that feeling when",
                        "thks" : "thank you",
                        "tho" : "though",
                        "thx" : "thank you",
                        "tia" : "thanks in advance",
                        "til" : "today i learned",
                        "tl;dr" : "too long i did not read",
                        "tldr" : "too long i did not read",
                        "tmb" : "tweet me back",
                        "tntl" : "trying not to laugh",
                        "ttyl" : "talk to you later",
                        "u" : "you",
                        "u2" : "you too",
                        "u4e" : "yours for ever",
                        "utc" : "coordinated universal time",
                        "w/" : "with",
                        "w/o" : "without",
                        "w8" : "wait",
                        "wassup" : "what is up",
                        "wb" : "welcome back",
                        "wtf" : "what the fuck",
                        "wtg" : "way to go",
                        "wtpa" : "where the party at",
                        "wuf" : "where are you from",
                        "wuzup" : "what is up",
                        "wywh" : "wish you were here",
                        "yd" : "yard",
                        "ygtr" : "you got that right",
                        "ynk" : "you never know",
                        "zzz" : "sleeping bored and tired"
                        }
            
        sample_typos_slang_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_typos_slang.keys()) + r')(?!\w)')
        sample_acronyms_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_acronyms.keys()) + r')(?!\w)')
        sample_abbr_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_abbr.keys()) + r')(?!\w)')
        
        text = sample_typos_slang_pattern.sub(lambda x: sample_typos_slang[x.group()], text)
        text = sample_acronyms_pattern.sub(lambda x: sample_acronyms[x.group()], text)
        text = sample_abbr_pattern.sub(lambda x: sample_abbr[x.group()], text)
        
        return text

In [30]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: informalAbbreviations_clean(x))

In [31]:
display(train_df.head())

,tweet,label,text_clean,tokenized
0,i don't like chatting with you.,0,i do not like chatting with you .,"[i, do, not, like, chatting, with, you, .]"
1,"you do not have friends, cause no body loves you.",1,"you do not have friends , because no body love...","[you, do, not, have, friends, ,, because, no, ..."
2,you don't deserve to exist.,1,you do not deserve to exist .,"[you, do, not, deserve, to, exist, .]"
3,no one cares about your opinion.,1,no one cares about your opinion .,"[no, one, cares, about, your, opinion, .]"
4,no one respect you.,1,no one respect you .,"[no, one, respect, you, .]"


### Remove non ascii charcters

In [32]:
def remove_non_ascii(text):

    """
        Remove non-ASCII characters 
    """

    return re.sub(r'[^\x00-\x7f]',r'', text) 

In [33]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_non_ascii(x))

### Remove any other special characters

In [34]:
def remove_special_characters(text):
    """
        Remove special special characters, including symbols, emojis, and other graphic characters
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [35]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_special_characters(x))

### Handling the negation

In [36]:
# def negate_sequence(text):
#     negation = False
#     delims = "?.,!:;"
#     result = []
#     words = text.split()
#     prev = None
#     pprev = None
#     for word in words:
#         stripped = word.strip(delims).lower()
#         negated = "not_" + stripped if negation else stripped
#         result.append(negated)

#         if any(neg in word for neg in ["not", "n't", "no",'none','neither','nor','never']):
#             negation = not negation

#         if any(c in word for c in delims):
#             negation = False

#     return result

In [37]:
# import nltk
# from nltk.corpus import wordnet

# def negation_function(x):   
#     synonyms = []
#     antonyms = []
#     for syn in wordnet.synsets(x):
#         for l in syn.lemmas():
#             synonyms.append(l.name())
#             if l.antonyms():
#                 antonyms.append(l.antonyms()[0].name())
               
#     if(len(antonyms) != 0):
#         return antonyms[0]
#     else:
#         return x

In [38]:
# def find_notWords(text):
#     neg = ["not", "n't", "no",'none','neither','nor','never']
#     if text in neg:
#         return "#"
#     else:
#         sub_str ="not_"
#         if (text.find(sub_str) != -1):
#             text = text.replace('not_','')
#             return negation_function(text) 
#         else:
#             return text

In [39]:
# def Handling_negation(text):
#     split_negation = negate_sequence(text)
#     full_text = []
#     for i in range(len(split_negation)):
#         full_text.append(find_notWords(split_negation[i]))
#     full_text = list(filter(lambda x: x != "#", full_text))
#     return full_text   
    

In [40]:
# train_df["text_clean"] = train_df["text_clean"].apply(lambda x: Handling_negation(x))
# train_df["text_clean"] = [' '.join(map(str, l)) for l in train_df["text_clean"]] 

In [41]:
display(train_df.head())

,tweet,label,text_clean,tokenized
0,i don't like chatting with you.,0,i do not like chatting with you .,"[i, do, not, like, chatting, with, you, .]"
1,"you do not have friends, cause no body loves you.",1,"you do not have friends , because no body love...","[you, do, not, have, friends, ,, because, no, ..."
2,you don't deserve to exist.,1,you do not deserve to exist .,"[you, do, not, deserve, to, exist, .]"
3,no one cares about your opinion.,1,no one cares about your opinion .,"[no, one, cares, about, your, opinion, .]"
4,no one respect you.,1,no one respect you .,"[no, one, respect, you, .]"


### Remove punctuation

In [42]:
def remove_punct(text):
    """
        Remove the punctuation
    """
    return text.translate(str.maketrans('', '', string.punctuation))

In [43]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_punct(x))

### Remove empty rows

In [44]:
train_df["text_clean"].replace('', np.nan, inplace=True)
train_df.dropna(subset=["text_clean"], inplace=True)


# Text Preprocessing


In [45]:
train_df.tail()

,tweet,label,text_clean,tokenized
10,Your mindset is not nice as you think it is.,1,your mindset is not nice as you think it is,"[Your, mindset, is, not, nice, as, you, think,..."
11,you are not ugly.,0,you are not ugly,"[you, are, not, ugly, .]"
12,you are not stuip.,1,you are not stuip,"[you, are, not, stuip, .]"
13,She turned to be not ugly as we thought she,1,she turned to be not ugly as we thought she,"[She, turned, to, be, not, ugly, as, we, thoug..."
14,We did not love her before but we do now.,1,we did not love her before but we do now,"[We, did, not, love, her, before, but, we, do,..."


### **Tokenization**

In [46]:

from nltk.tokenize import word_tokenize

train_df["tokenized"] = train_df["text_clean"].apply(word_tokenize)
train_df.head()

,tweet,label,text_clean,tokenized
0,i don't like chatting with you.,0,i do not like chatting with you,"[i, do, not, like, chatting, with, you]"
1,"you do not have friends, cause no body loves you.",1,you do not have friends because no body loves...,"[you, do, not, have, friends, because, no, bod..."
2,you don't deserve to exist.,1,you do not deserve to exist,"[you, do, not, deserve, to, exist]"
3,no one cares about your opinion.,1,no one cares about your opinion,"[no, one, cares, about, your, opinion]"
4,no one respect you.,1,no one respect you,"[no, one, respect, you]"



### **Remove stop word without the negation**

In [47]:
# My_stopWordList=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't" ]

In [48]:
stop = set(stopwords.words('english'))

In [49]:
nltk.download("stopwords")
from nltk.corpus import stopwords


train_df['stopwords_removed'] = train_df["tokenized"].apply(lambda x: [word for word in x if word not in stop ])

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


In [50]:
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()

### **Part of Speech Tagging (POS Tagging):**

Part of speech tagging (POS tagging) distinguishes the part of speech (noun, verb, adjective, and etc.) of each word in the text.

In [51]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import brown

wordnet_map = {"N":wordnet.NOUN, 
               "V":wordnet.VERB, 
               "J":wordnet.ADJ, 
               "R":wordnet.ADV
              }
    
train_sents = brown.tagged_sents(categories='news')
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)

def pos_tag_wordnet(text, pos_tag_type="pos_tag"):
    """
        Create pos_tag with wordnet format
    """
    pos_tagged_text = t2.tag(text)
    
    # map the pos tagging output with wordnet output 
    pos_tagged_text = [(word, wordnet_map.get(pos_tag[0])) if pos_tag[0] in wordnet_map.keys() else (word, wordnet.NOUN) for (word, pos_tag) in pos_tagged_text ]
    return pos_tagged_text

In [52]:
train_df['pos_tag'] = train_df['stopwords_removed'].apply(lambda x: pos_tag_wordnet(x))

### **Lemmatization:**

Lemmatization is the task of determining that two words have the same root, despite their surface differences. The words am, are, and is have the shared lemma be; the words dinner and dinners both have the lemma dinner. 
Lemmatizing each of these forms to the same lemma will let us ﬁnd all mentions of words in Russian like Moscow.

In [53]:
from nltk.stem import WordNetLemmatizer

def lemmatize_word(text):
    """
        Lemmatize the tokenized words
    """

    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma

In [54]:
# from nltk.stem import WordNetLemmatizer as form_replacer
# lemmatized_irr2 = list(set([form_replacer().lemmatize(x) for x in ['isis','one']]))
# print(lemmatized_irr2 )
# #apply(lambda x: [lemmatizer.lemmatize(word) for word in x)

In [55]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

train_df['lemmatize_word_w_pos'] = train_df['pos_tag'].apply(lambda x: lemmatize_word(x))
train_df['lemmatize_text'] = [' '.join(map(str, l)) for l in train_df['lemmatize_word_w_pos']] # join back to text

### Double check that there is not any empty row after cleaning

In [56]:
train_df['lemmatize_text'].replace('', np.nan, inplace=True)
train_df.dropna(subset=['lemmatize_text'], inplace=True)

### **Saving Dataset**

In [57]:
train_df['lemmatize_text'].to_csv('tryyyyyyyyyyyyyyyy.csv')# choose the name to be saved with 

In [58]:
# !pip install textblob

In [59]:
# from textblob import TextBlob

In [60]:
# from textblob import TextBlob

# # Preparing an input sentence
# sentence = 'i do dislike chatting with you'

# analysisPol = TextBlob(sentence).polarity


# print(analysisPol)